In [129]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression, SGDRegressor, SGDClassifier, Lasso, Ridge

import sklearn
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from sklearn.feature_selection import SelectKBest

from lightgbm import LGBMRegressor

from catboost import CatBoostRegressor

import warnings
warnings.filterwarnings('ignore')

In [113]:
X = pd.read_csv('data/features_data_without_ld.csv', index_col=0)
y = pd.read_csv('data/target_data', index_col=0)
X.shape

(5798, 49)

In [114]:
X_clean = X[~X.isna_Al2O3].drop(['isna_Al2O3', 'isna_SiO2'], axis=1)
y_clean = y[~y.isna_Al2O3].drop(['isna_Al2O3', 'isna_SiO2'], axis=1)

In [115]:
X_clean.shape

(4369, 47)

In [116]:
results_mape = pd.DataFrame(index = ['химшлак последний Al2O3', 'химшлак последний CaO',
       'химшлак последний FeO', 'химшлак последний MgO',
       'химшлак последний MnO', 'химшлак последний R',
       'химшлак последний SiO2'])

In [148]:
def model_results(X, y, model, name_model):
    global results_mape
    results_mape[name_model] = None
    for name in y.columns:
        mape = cross_val_score(model, X, y[name], cv=5, scoring='neg_mean_absolute_percentage_error')
        print(name, 'mape:', round(-mape.mean() * 100, 2))
        results_mape[name_model][name] = round(-mape.mean() * 100, 2)    
        
def model_results_with_grid_cv(X, y, model, name_model, parameters_grid,):
    global results_mape
    results_mape[name_model] = None
    for name in y.columns:
        grid_cv = GridSearchCV(model, param_grid=parameters_grid, cv=5)
        grid_cv.fit(X_clean, y_clean[name])
#         print("Best score: " + str(grid_cv.best_score_))
        print("Patameters: ", grid_cv.best_params_)
        model = grid_cv.best_estimator_
        mape = cross_val_score(model, X, y[name], cv=5, scoring='neg_mean_absolute_percentage_error')
        print(name, 'mape:', round(-mape.mean() * 100, 2))
        results_mape[name_model][name] = round(-mape.mean() * 100, 2)
    
def model_results_with_random_search_cv(X, y, model, name_model, param_grid,):
    global results_mape
    results_mape[name_model] = None
    for name in y.columns:
        random_cv = RandomizedSearchCV(model, param_distributions=param_grid, cv=5, n_jobs=-1)
        random_cv.fit(X_clean, y_clean[name])
#         print("Best score: " + str(grid_cv.best_score_))
        print("Patameters: ", grid_cv.best_params_)
        model = random_cv.best_estimator_
        mape = cross_val_score(model, X, y[name], cv=5, scoring='neg_mean_absolute_percentage_error')
        print(name, 'mape:', round(-mape.mean() * 100, 2))
        results_mape[name_model][name] = round(-mape.mean() * 100, 2)

In [118]:
# def model_results_2_step(X, y, model, name_model ):
#     global results_mape
#     results_mape[name_model] = None
    
#     for name in ['химшлак последний SiO2', 'химшлак последний CaO', 'химшлак последний R']:
#         mape = cross_val_score(model, X, y[name], cv=5, scoring='neg_mean_absolute_percentage_error')
#         print(name, 'mape:', round(-mape.mean() * 100, 2))
#         results_mape[name_model][name] = round(-mape.mean() * 100, 2)
        
#     X_with_3_target = X.copy().merge(y[['химшлак последний SiO2', 'химшлак последний CaO', 'химшлак последний R']],\
#                                     how='right', left_index=True, right_index=True)
    
#     for name in ['химшлак последний Al2O3', 'химшлак последний FeO', 'химшлак последний MgO',
#        'химшлак последний MnO']:
#         mape = cross_val_score(model, X_with_3_target, y[name], cv=5, scoring='neg_mean_absolute_percentage_error')
#         print(name, 'mape:', round(-mape.mean() * 100, 2))
#         results_mape[name_model][name] = round(-mape.mean() * 100, 2)

In [160]:
def model_results_features_with_target_data(X, y, model, name_model ):
    global results_mape
    results_mape[name_model] = None
    
    for name in y.columns:
        X_with_targets = X.merge(y.drop(name, axis=1), how='right', left_index=True, right_index=True)
        mape = cross_val_score(model, X_with_targets, y[name], cv=5, scoring='neg_mean_absolute_percentage_error')
        print(name, 'mape:', round(-mape.mean() * 100, 2))
        results_mape[name_model][name] = round(-mape.mean() * 100, 2)
        
def model_results_with_target_data_random_cv(X, y, model, name_model, param_grid):
    global results_mape
    results_mape[name_model] = None
    for name in y.columns:
        X_with_targets = X.merge(y.drop(name, axis=1), how='right', left_index=True, right_index=True)
        random_cv = RandomizedSearchCV(model, param_distributions=param_grid, cv=5)
        random_cv.fit(X_with_targets, y[name])
#         print("Best score: " + str(grid_cv.best_score_))
        print("Patameters: ", random_cv.best_params_)
        model = random_cv.best_estimator_
        mape = cross_val_score(model, X_with_targets, y[name], cv=5, scoring='neg_mean_absolute_percentage_error')
        print(name, 'mape:', round(-mape.mean() * 100, 2))
        results_mape[name_model][name] = round(-mape.mean() * 100, 2)        

### Модели на данных без пропущенных значений SiO2 и Al2O3

In [120]:
print('Линейная регрессия')
model_results(X_clean, y_clean, LinearRegression(), 'LinearReg')

Линейная регрессия
химшлак последний Al2O3 mape: 11.09
химшлак последний CaO mape: 4.84
химшлак последний FeO mape: 35.75
химшлак последний MgO mape: 25.92
химшлак последний MnO mape: 29.84
химшлак последний R mape: 6.17
химшлак последний SiO2 mape: 6.98


In [121]:
# model_results_2_step(X_clean, y_clean, LinearRegression(),'LinearReg_2step')

In [122]:
model_results_features_with_target_data(X_clean, y_clean, LinearRegression(),'LinearReg_features_with_target')

химшлак последний Al2O3 mape: 10.11
химшлак последний CaO mape: 1.5
химшлак последний FeO mape: 33.37
химшлак последний MgO mape: 23.34
химшлак последний MnO mape: 24.1
химшлак последний R mape: 1.54
химшлак последний SiO2 mape: 1.5


In [123]:
results_mape

,LinearReg,LinearReg_features_with_target
химшлак последний Al2O3,11.09,10.11
химшлак последний CaO,4.84,1.5
химшлак последний FeO,35.75,33.37
химшлак последний MgO,25.92,23.34
химшлак последний MnO,29.84,24.1
химшлак последний R,6.17,1.54
химшлак последний SiO2,6.98,1.5


In [150]:
print('Lasso')
model_results(X_clean, y_clean, Lasso(), 'Lasso')
print('\nRidge features with targets')
model_results_features_with_target_data(X_clean, y_clean, Lasso(),'Lasso_features_with_target')

Lasso
химшлак последний Al2O3 mape: 16.44
химшлак последний CaO mape: 5.25
химшлак последний FeO mape: 36.31
химшлак последний MgO mape: 31.1
химшлак последний MnO mape: 35.25
химшлак последний R mape: 8.21
химшлак последний SiO2 mape: 8.27

Ridge features with targets
химшлак последний Al2O3 mape: 16.44
химшлак последний CaO mape: 4.56
химшлак последний FeO mape: 36.31
химшлак последний MgO mape: 27.87
химшлак последний MnO mape: 35.25
химшлак последний R mape: 8.21
химшлак последний SiO2 mape: 7.74


In [132]:
print('Ridge')
model_results(X_clean, y_clean, Ridge(), 'Ridge')
print('\nRidge features with targets')
model_results_features_with_target_data(X_clean, y_clean, SGDRegressor(),'SGDRegressor_features_with_target')

Ridge
химшлак последний Al2O3 mape: 11.09
химшлак последний CaO mape: 4.84
химшлак последний FeO mape: 35.74
химшлак последний MgO mape: 25.93
химшлак последний MnO mape: 29.86
химшлак последний R mape: 6.17
химшлак последний SiO2 mape: 6.98

Ridge features with targets
химшлак последний Al2O3 mape: 16156395377.31
химшлак последний CaO mape: 35523944.22
химшлак последний FeO mape: 107323201335.0
химшлак последний MgO mape: 5614305092.71
химшлак последний MnO mape: 418473378281.03
химшлак последний R mape: 22944131727.36
химшлак последний SiO2 mape: 1515363493.73


In [131]:
print('SGDRegressor')
model_results(X_clean, y_clean, SGDRegressor(), 'SGDRegressor')
print('\nSGDRegressor features with targets')
model_results_features_with_target_data(X_clean, y_clean, Ridge(),'Ridge_features_with_target')

SGDRegressor
химшлак последний Al2O3 mape: 11.15
химшлак последний CaO mape: 4.91
химшлак последний FeO mape: 35.98
химшлак последний MgO mape: 26.15
химшлак последний MnO mape: 31.02
химшлак последний R mape: 6.3
химшлак последний SiO2 mape: 7.09

SGDRegressor features with targets
химшлак последний Al2O3 mape: 10.11
химшлак последний CaO mape: 1.51
химшлак последний FeO mape: 33.55
химшлак последний MgO mape: 23.36
химшлак последний MnO mape: 24.09
химшлак последний R mape: 1.54
химшлак последний SiO2 mape: 1.5


In [133]:
print('RandomForest')
model_results(X_clean, y_clean, RandomForestRegressor(), 'RandomForestReg' )

RandomForest
химшлак последний Al2O3 mape: 11.48
химшлак последний CaO mape: 4.89
химшлак последний FeO mape: 36.7
химшлак последний MgO mape: 26.79
химшлак последний MnO mape: 33.16
химшлак последний R mape: 6.2
химшлак последний SiO2 mape: 7.12


In [ ]:
# очень большая ошибка


# print('SGDRegressor')
# param_grid = {
#     'alpha': 10.0 ** -np.arange(1, 7),
#     'loss': ['squared_loss', 'huber', 'epsilon_insensitive'],
#     'penalty': ['l2', 'l1']
# }
# model_results_with_grid_cv(X_clean, y_clean, SGDRegressor(), 'SGDReg_with_tun', parameters_grid=param_grid)        

In [141]:
print('LGBMRegressor')
model_results(X_clean, y_clean, LGBMRegressor(), 'LGBMRegressor')

LGBMRegressor
химшлак последний Al2O3 mape: 11.56
химшлак последний CaO mape: 5.0
химшлак последний FeO mape: 36.63
химшлак последний MgO mape: 26.73
химшлак последний MnO mape: 32.29
химшлак последний R mape: 6.17
химшлак последний SiO2 mape: 7.1


In [151]:
param_grid = {
    'num_leaves': [ 28, 31, 50],
    'max_depth': [-1, 3, 5],
    'n_estimators': [100, 200, 500],
}
# model_results_with_grid_cv(X_clean, y_clean, LGBMRegressor(), 'LGBMRegressor_witn_tun', parameters_grid=param_grid,) 

model_results(X_clean, y_clean, LGBMRegressor(max_depth=3, num_leaves=28, n_estimators=100), 'LGBMRegressor_witn_tun')


химшлак последний Al2O3 mape: 11.26
химшлак последний CaO mape: 4.9
химшлак последний FeO mape: 36.19
химшлак последний MgO mape: 26.6
химшлак последний MnO mape: 31.62
химшлак последний R mape: 6.16
химшлак последний SiO2 mape: 7.03


In [ ]:
# results_mape.to_csv('model_results')

In [144]:
print('CatBoostRegressor')
model_results(X_clean, y_clean, CatBoostRegressor(verbose=False), 'CatBoostRegressor')
print('\nCatBoostRegressor features with targets')
model_results_features_with_target_data(X_clean, y_clean, CatBoostRegressor(verbose=False),'CatBoostRegresso_features_with_target')

CatBoostRegressor
химшлак последний Al2O3 mape: 11.34
химшлак последний CaO mape: 4.97
химшлак последний FeO mape: 36.51
химшлак последний MgO mape: 26.55
химшлак последний MnO mape: 31.94
химшлак последний R mape: 6.14
химшлак последний SiO2 mape: 7.02

CatBoostRegressor features with targets
химшлак последний Al2O3 mape: 9.36
химшлак последний CaO mape: 1.37
химшлак последний FeO mape: 31.82
химшлак последний MgO mape: 19.58
химшлак последний MnO mape: 24.05
химшлак последний R mape: 1.42
химшлак последний SiO2 mape: 1.44


In [153]:
param_grid = {'depth': [6,8,10],
            'learning_rate' : [0.01, 0.05, 0.1],
            'iterations'    : [30, 50, 100]
                 }

model_results_with_grid_cv(X_clean, y_clean, CatBoostRegressor(verbose=False), 'CatBoostRegressor_with_tun', parameters_grid=param_grid) 

In [195]:
param_grid = {'depth': [6,8,10],
            'learning_rate' : [0.01, 0.05, 0.1],
            'iterations'    : [30, 50, 100]
                 }

# model_results_with_random_search_cv(X_clean, y_clean, CatBoostRegressor(verbose=False), 'CatBoostRegressor_with_tun', param_grid) 

In [ ]:
param_grid = {'depth': [6,8,10],
            'learning_rate' : [0.01, 0.05, 0.1],
            'iterations'    : [30, 50, 100]
                 }

model_results_with_target_data_random_cv(X_clean, y_clean, CatBoostRegressor(verbose=False), 'CatBoostRegressor_features_with_target_tun', param_grid) 

In [ ]:
# param_grid = {
#     'alpha': 10.0 ** -np.arange(1, 7),
#     'loss': ['squared_loss', 'huber', 'epsilon_insensitive'],
#     'penalty': ['l2', 'l1']
# }
# model_results_with_ra_cv(X_clean, y_clean, SGDRegressor(), 'SGDReg_with_tun', parameters_grid=param_grid)        

In [168]:
param_grid = {'n_estimators': [100, 300, 500],
                  'max_depth': [1, 3,  5],
                 'max_features': ['auto', 'sqrt', 'log2']}

model_results_with_target_data_random_cv(X_clean, y_clean, RandomForestRegressor(), 'RandomForestReg_witn_tun', param_grid) 
# results_mape['RandomForestReg_witn_tun'] = [11.48, 5.0, 35.0, 28.8, 31.67, 6, 7.02]

Patameters:  {'n_estimators': 500, 'max_features': 'auto', 'max_depth': 5}
химшлак последний Al2O3 mape: 10.14
Patameters:  {'n_estimators': 500, 'max_features': 'auto', 'max_depth': 5}
химшлак последний CaO mape: 2.13
Patameters:  {'n_estimators': 100, 'max_features': 'auto', 'max_depth': 3}
химшлак последний FeO mape: 33.05
Patameters:  {'n_estimators': 100, 'max_features': 'auto', 'max_depth': 5}
химшлак последний MgO mape: 21.06
Patameters:  {'n_estimators': 500, 'max_features': 'auto', 'max_depth': 5}
химшлак последний MnO mape: 25.67
Patameters:  {'n_estimators': 100, 'max_features': 'auto', 'max_depth': 5}
химшлак последний R mape: 1.73
Patameters:  {'n_estimators': 300, 'max_features': 'auto', 'max_depth': 5}
химшлак последний SiO2 mape: 1.92


In [177]:
results_mape.to_excel('model_selection_mape.xlsx')

In [185]:
results_mape.T.min()

химшлак последний Al2O3     9.36
химшлак последний CaO       1.37
химшлак последний FeO      31.82
химшлак последний MgO      19.58
химшлак последний MnO      24.05
химшлак последний R         1.42
химшлак последний SiO2      1.44
dtype: float64

In [196]:
results_mape.T.columns

Index(['химшлак последний Al2O3', 'химшлак последний CaO',
       'химшлак последний FeO', 'химшлак последний MgO',
       'химшлак последний MnO', 'химшлак последний R',
       'химшлак последний SiO2'],
      dtype='object')

# Предсказание на всех признаках

In [197]:
X = pd.read_csv('data/features_data.csv', index_col=0)
y = pd.read_csv('data/target_data', index_col=0)
X.shape

(5798, 64)

In [198]:
X_clean = X[~X.isna_Al2O3].drop(['isna_Al2O3', 'isna_SiO2'], axis=1)
y_clean = y[~y.isna_Al2O3].drop(['isna_Al2O3', 'isna_SiO2'], axis=1)

In [199]:
X_clean.shape

(4369, 62)

In [200]:
results_mape = pd.DataFrame(index = ['химшлак последний Al2O3', 'химшлак последний CaO',
       'химшлак последний FeO', 'химшлак последний MgO',
       'химшлак последний MnO', 'химшлак последний R',
       'химшлак последний SiO2'])

In [201]:
def model_results(X, y, model, name_model):
    global results_mape
    results_mape[name_model] = None
    for name in y.columns:
        mape = cross_val_score(model, X, y[name], cv=5, scoring='neg_mean_absolute_percentage_error')
        print(name, 'mape:', round(-mape.mean() * 100, 2))
        results_mape[name_model][name] = round(-mape.mean() * 100, 2)    
        
def model_results_with_grid_cv(X, y, model, name_model, parameters_grid,):
    global results_mape
    results_mape[name_model] = None
    for name in y.columns:
        grid_cv = GridSearchCV(model, param_grid=parameters_grid, cv=5)
        grid_cv.fit(X_clean, y_clean[name])
#         print("Best score: " + str(grid_cv.best_score_))
        print("Patameters: ", grid_cv.best_params_)
        model = grid_cv.best_estimator_
        mape = cross_val_score(model, X, y[name], cv=5, scoring='neg_mean_absolute_percentage_error')
        print(name, 'mape:', round(-mape.mean() * 100, 2))
        results_mape[name_model][name] = round(-mape.mean() * 100, 2)
    
def model_results_with_random_search_cv(X, y, model, name_model, param_grid,):
    global results_mape
    results_mape[name_model] = None
    for name in y.columns:
        random_cv = RandomizedSearchCV(model, param_distributions=param_grid, cv=5, n_jobs=-1)
        random_cv.fit(X_clean, y_clean[name])
#         print("Best score: " + str(grid_cv.best_score_))
        print("Patameters: ", grid_cv.best_params_)
        model = random_cv.best_estimator_
        mape = cross_val_score(model, X, y[name], cv=5, scoring='neg_mean_absolute_percentage_error')
        print(name, 'mape:', round(-mape.mean() * 100, 2))
        results_mape[name_model][name] = round(-mape.mean() * 100, 2)

In [202]:
# def model_results_2_step(X, y, model, name_model ):
#     global results_mape
#     results_mape[name_model] = None
    
#     for name in ['химшлак последний SiO2', 'химшлак последний CaO', 'химшлак последний R']:
#         mape = cross_val_score(model, X, y[name], cv=5, scoring='neg_mean_absolute_percentage_error')
#         print(name, 'mape:', round(-mape.mean() * 100, 2))
#         results_mape[name_model][name] = round(-mape.mean() * 100, 2)
        
#     X_with_3_target = X.copy().merge(y[['химшлак последний SiO2', 'химшлак последний CaO', 'химшлак последний R']],\
#                                     how='right', left_index=True, right_index=True)
    
#     for name in ['химшлак последний Al2O3', 'химшлак последний FeO', 'химшлак последний MgO',
#        'химшлак последний MnO']:
#         mape = cross_val_score(model, X_with_3_target, y[name], cv=5, scoring='neg_mean_absolute_percentage_error')
#         print(name, 'mape:', round(-mape.mean() * 100, 2))
#         results_mape[name_model][name] = round(-mape.mean() * 100, 2)

In [203]:
def model_results_features_with_target_data(X, y, model, name_model ):
    global results_mape
    results_mape[name_model] = None
    
    for name in y.columns:
        X_with_targets = X.merge(y.drop(name, axis=1), how='right', left_index=True, right_index=True)
        mape = cross_val_score(model, X_with_targets, y[name], cv=5, scoring='neg_mean_absolute_percentage_error')
        print(name, 'mape:', round(-mape.mean() * 100, 2))
        results_mape[name_model][name] = round(-mape.mean() * 100, 2)
        
def model_results_with_target_data_random_cv(X, y, model, name_model, param_grid):
    global results_mape
    results_mape[name_model] = None
    for name in y.columns:
        X_with_targets = X.merge(y.drop(name, axis=1), how='right', left_index=True, right_index=True)
        random_cv = RandomizedSearchCV(model, param_distributions=param_grid, cv=5)
        random_cv.fit(X_with_targets, y[name])
#         print("Best score: " + str(grid_cv.best_score_))
        print("Patameters: ", random_cv.best_params_)
        model = random_cv.best_estimator_
        mape = cross_val_score(model, X_with_targets, y[name], cv=5, scoring='neg_mean_absolute_percentage_error')
        print(name, 'mape:', round(-mape.mean() * 100, 2))
        results_mape[name_model][name] = round(-mape.mean() * 100, 2)        

### Модели на данных без пропущенных значений SiO2 и Al2O3

In [204]:
print('Линейная регрессия')
model_results(X_clean, y_clean, LinearRegression(), 'LinearReg')

Линейная регрессия
химшлак последний Al2O3 mape: 11.12
химшлак последний CaO mape: 4.84
химшлак последний FeO mape: 35.75
химшлак последний MgO mape: 25.93
химшлак последний MnO mape: 29.58
химшлак последний R mape: 6.17
химшлак последний SiO2 mape: 7.0


In [205]:
# model_results_2_step(X_clean, y_clean, LinearRegression(),'LinearReg_2step')

In [206]:
model_results_features_with_target_data(X_clean, y_clean, LinearRegression(),'LinearReg_features_with_target')

химшлак последний Al2O3 mape: 10.17
химшлак последний CaO mape: 1.51
химшлак последний FeO mape: 33.39
химшлак последний MgO mape: 23.41
химшлак последний MnO mape: 24.09
химшлак последний R mape: 1.54
химшлак последний SiO2 mape: 1.5


In [207]:
results_mape

,LinearReg,LinearReg_features_with_target
химшлак последний Al2O3,11.12,10.17
химшлак последний CaO,4.84,1.51
химшлак последний FeO,35.75,33.39
химшлак последний MgO,25.93,23.41
химшлак последний MnO,29.58,24.09
химшлак последний R,6.17,1.54
химшлак последний SiO2,7,1.5


In [208]:
print('Lasso')
model_results(X_clean, y_clean, Lasso(), 'Lasso')
print('\nRidge features with targets')
model_results_features_with_target_data(X_clean, y_clean, Lasso(),'Lasso_features_with_target')

Lasso
химшлак последний Al2O3 mape: 16.44
химшлак последний CaO mape: 5.25
химшлак последний FeO mape: 36.31
химшлак последний MgO mape: 31.1
химшлак последний MnO mape: 35.25
химшлак последний R mape: 8.21
химшлак последний SiO2 mape: 8.27

Ridge features with targets
химшлак последний Al2O3 mape: 16.44
химшлак последний CaO mape: 4.56
химшлак последний FeO mape: 36.31
химшлак последний MgO mape: 27.87
химшлак последний MnO mape: 35.25
химшлак последний R mape: 8.21
химшлак последний SiO2 mape: 7.74


In [209]:
print('Ridge')
model_results(X_clean, y_clean, Ridge(), 'Ridge')
print('\nRidge features with targets')
model_results_features_with_target_data(X_clean, y_clean, SGDRegressor(),'SGDRegressor_features_with_target')

Ridge
химшлак последний Al2O3 mape: 11.09
химшлак последний CaO mape: 4.83
химшлак последний FeO mape: 35.72
химшлак последний MgO mape: 25.93
химшлак последний MnO mape: 29.59
химшлак последний R mape: 6.17
химшлак последний SiO2 mape: 6.99

Ridge features with targets
химшлак последний Al2O3 mape: 16546190169.34
химшлак последний CaO mape: 61885260.43
химшлак последний FeO mape: 147452820034.46
химшлак последний MgO mape: 10989986693.84
химшлак последний MnO mape: 595894197068.36
химшлак последний R mape: 56394157153.14
химшлак последний SiO2 mape: 2076775034.61


In [210]:
print('SGDRegressor')
model_results(X_clean, y_clean, SGDRegressor(), 'SGDRegressor')
print('\nSGDRegressor features with targets')
model_results_features_with_target_data(X_clean, y_clean, Ridge(),'Ridge_features_with_target')

SGDRegressor
химшлак последний Al2O3 mape: 11.32
химшлак последний CaO mape: 4.89
химшлак последний FeO mape: 36.1
химшлак последний MgO mape: 26.04
химшлак последний MnO mape: 30.04
химшлак последний R mape: 6.33
химшлак последний SiO2 mape: 7.01

SGDRegressor features with targets
химшлак последний Al2O3 mape: 10.15
химшлак последний CaO mape: 1.52
химшлак последний FeO mape: 33.54
химшлак последний MgO mape: 23.41
химшлак последний MnO mape: 24.06
химшлак последний R mape: 1.54
химшлак последний SiO2 mape: 1.5


In [211]:
print('RandomForest')
model_results(X_clean, y_clean, RandomForestRegressor(), 'RandomForestReg' )

RandomForest
химшлак последний Al2O3 mape: 11.37
химшлак последний CaO mape: 4.91
химшлак последний FeO mape: 36.43
химшлак последний MgO mape: 26.84
химшлак последний MnO mape: 33.51
химшлак последний R mape: 6.21
химшлак последний SiO2 mape: 7.12


In [212]:
# очень большая ошибка


# print('SGDRegressor')
# param_grid = {
#     'alpha': 10.0 ** -np.arange(1, 7),
#     'loss': ['squared_loss', 'huber', 'epsilon_insensitive'],
#     'penalty': ['l2', 'l1']
# }
# model_results_with_grid_cv(X_clean, y_clean, SGDRegressor(), 'SGDReg_with_tun', parameters_grid=param_grid)        

In [213]:
print('LGBMRegressor')
model_results(X_clean, y_clean, LGBMRegressor(), 'LGBMRegressor')

LGBMRegressor
химшлак последний Al2O3 mape: 11.42
химшлак последний CaO mape: 5.0
химшлак последний FeO mape: 36.66
химшлак последний MgO mape: 26.63
химшлак последний MnO mape: 32.07
химшлак последний R mape: 6.16
химшлак последний SiO2 mape: 7.14


In [214]:
param_grid = {
    'num_leaves': [ 28, 31, 50],
    'max_depth': [-1, 3, 5],
    'n_estimators': [100, 200, 500],
}
# model_results_with_grid_cv(X_clean, y_clean, LGBMRegressor(), 'LGBMRegressor_witn_tun', parameters_grid=param_grid,) 

model_results(X_clean, y_clean, LGBMRegressor(max_depth=3, num_leaves=28, n_estimators=100), 'LGBMRegressor_witn_tun')


химшлак последний Al2O3 mape: 11.25
химшлак последний CaO mape: 4.92
химшлак последний FeO mape: 36.13
химшлак последний MgO mape: 26.5
химшлак последний MnO mape: 31.3
химшлак последний R mape: 6.15
химшлак последний SiO2 mape: 7.03


In [215]:
# results_mape.to_csv('model_results')

In [216]:
print('CatBoostRegressor')
model_results(X_clean, y_clean, CatBoostRegressor(verbose=False), 'CatBoostRegressor')
print('\nCatBoostRegressor features with targets')
model_results_features_with_target_data(X_clean, y_clean, CatBoostRegressor(verbose=False),'CatBoostRegresso_features_with_target')

CatBoostRegressor
химшлак последний Al2O3 mape: 11.33
химшлак последний CaO mape: 5.0
химшлак последний FeO mape: 36.36
химшлак последний MgO mape: 26.51
химшлак последний MnO mape: 31.77
химшлак последний R mape: 6.09
химшлак последний SiO2 mape: 7.09

CatBoostRegressor features with targets
химшлак последний Al2O3 mape: 9.42
химшлак последний CaO mape: 1.37
химшлак последний FeO mape: 31.79
химшлак последний MgO mape: 19.59
химшлак последний MnO mape: 24.14
химшлак последний R mape: 1.42
химшлак последний SiO2 mape: 1.43


In [217]:
param_grid = {'depth': [6,8,10],
            'learning_rate' : [0.01, 0.05, 0.1],
            'iterations'    : [30, 50, 100]
                 }

model_results_with_grid_cv(X_clean, y_clean, CatBoostRegressor(verbose=False), 'CatBoostRegressor_with_tun', parameters_grid=param_grid) 

KeyboardInterrupt: 

In [ ]:
param_grid = {'depth': [6,8,10],
            'learning_rate' : [0.01, 0.05, 0.1],
            'iterations'    : [30, 50, 100]
                 }

# model_results_with_random_search_cv(X_clean, y_clean, CatBoostRegressor(verbose=False), 'CatBoostRegressor_with_tun', param_grid) 

In [ ]:
param_grid = {'depth': [6,8,10],
            'learning_rate' : [0.01, 0.05, 0.1],
            'iterations'    : [30, 50, 100]
                 }

model_results_with_target_data_random_cv(X_clean, y_clean, CatBoostRegressor(verbose=False), 'CatBoostRegressor_features_with_target_tun', param_grid) 

In [ ]:
# param_grid = {
#     'alpha': 10.0 ** -np.arange(1, 7),
#     'loss': ['squared_loss', 'huber', 'epsilon_insensitive'],
#     'penalty': ['l2', 'l1']
# }
# model_results_with_ra_cv(X_clean, y_clean, SGDRegressor(), 'SGDReg_with_tun', parameters_grid=param_grid)        

In [ ]:
param_grid = {'n_estimators': [100, 300, 500],
                  'max_depth': [1, 3,  5],
                 'max_features': ['auto', 'sqrt', 'log2']}

model_results_with_target_data_random_cv(X_clean, y_clean, RandomForestRegressor(), 'RandomForestReg_witn_tun', param_grid) 
# results_mape['RandomForestReg_witn_tun'] = [11.48, 5.0, 35.0, 28.8, 31.67, 6, 7.02]

In [218]:
results_mape.to_excel('model_selection_mape_all_line.xlsx')

In [219]:
results_mape.T

,химшлак последний Al2O3,химшлак последний CaO,химшлак последний FeO,химшлак последний MgO,химшлак последний MnO,химшлак последний R,химшлак последний SiO2
LinearReg,11.12,4.84,35.75,25.93,29.58,6.17,7
LinearReg_features_with_target,10.17,1.51,33.39,23.41,24.09,1.54,1.5
Lasso,16.44,5.25,36.31,31.1,35.25,8.21,8.27
Lasso_features_with_target,16.44,4.56,36.31,27.87,35.25,8.21,7.74
Ridge,11.09,4.83,35.72,25.93,29.59,6.17,6.99
SGDRegressor_features_with_target,1.65462e+10,6.18853e+07,1.47453e+11,1.099e+10,5.95894e+11,5.63942e+10,2.07678e+09
SGDRegressor,11.32,4.89,36.1,26.04,30.04,6.33,7.01
Ridge_features_with_target,10.15,1.52,33.54,23.41,24.06,1.54,1.5
RandomForestReg,11.37,4.91,36.43,26.84,33.51,6.21,7.12
LGBMRegressor,11.42,5,36.66,26.63,32.07,6.16,7.14
